## 3.2.3 BM25를 활용 한 RAG System

In [1]:
! pip install llama-index-retrievers-bm25
! pip install konlpy

### 1. 데이터 준비

In [2]:
### Step1. 데이터 로드
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()
print(f'문서의 총 개수 : {len(documents)}')

문서의 총 개수 : 83


In [2]:
from konlpy.tag import Okt
# Okt 형태소 분석기 초기화
okt = Okt()

# 문서 내용 토큰화 함수 정의
def tokenize_korean_text(text):
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화
tokenize_korean_text('이 책은 RAG 시스템을 구축하는 방법에 대해 다룹니다.')

['이', '책', '은', 'RAG', '시스템', '을', '구축', '하는', '방법', '에', '대해', '다룹니다', '.']

In [8]:
### Step 2. 청킹 및 인덱스 생성
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)
print(f'노드의 개수 : {len(nodes)}')

노드의 개수 : 156


In [ ]:
# BM25 기반 인덱스 생성
from llama_index.core.indices.keyword_table import KeywordTableIndex
index = KeywordTableIndex.from_documents(documents=documents,
                                         text_splitter=splitter,
                                         extract_keyword=tokenize_korean_text,
                                         max_keywords_per_node=10, # 각 청크에서 추출 된 상위 N개 키워드 선택
                                         max_nodes_per_query=10, # 쿼리에 대해 반환 할 노드 수를 제한
                                         show_progress=True # 진행 상황 확인
                                        )

Parsing nodes:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting keywords from nodes: 100%|██████████| 42/42 [00:49<00:00,  1.17s/it]


In [ ]:
# 인덱스 로컬 경로 저장
index.storage_context.persist(persist_dir="./index/ch03_bm25_index")

In [6]:
### Step6-2. 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch03_bm25_index") 
index = load_index_from_storage(storage_context)

### 2. BM25 Retriever 객체 생성

In [ ]:
# custom bm25
import bm25s
from llama_index.retrievers.bm25 import BM25Retriever
corpus_tokens = bm25s.tokenize(
                [node.text for node in nodes]
)
custom_bm25 = bm25s.BM25(k1=0.5, b=0.75) # BM25 파라미터 조정
custom_bm25.index(corpus_tokens, show_progress=True)

bm25_retriever_obj = BM25Retriever(existing_bm25=custom_bm25)

In [10]:
bm25_retriever = bm25_retriever_obj.from_defaults(
    index=index,          # BM25를 적용할 인덱스
    similarity_top_k=2,   # 유사도 기반 상위 k개 결과 반환

)
# from llama_index.retrievers.bm25 import BM25Retriever
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=2,
# )

In [11]:
from llama_index.core.response.notebook_utils import display_source_node
# will retrieve context from specific companies
query = '변동금리'
retrieved_nodes = bm25_retriever.retrieve(query)
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** c128600e-9d92-4cea-bd8b-7ca84241e6bb<br>**Similarity:** 2.464564323425293<br>**Text:** 국채선물의 경우 거래증거금률이 매우 낮아(3년 만기, 0.5%) 
레버리지가 매우 높으므로 헤지펀드 등 외국인들이 이런 투기적 거래를 많이 이용하고 
있으며 국채선물가격에 미치는 영향도 상당히 크다. 한편 금리선물시장에서의 가격 움직
임을 보고 향후 시장금리가 어떤 방향으로 움직일 것인지를 예측하기도 하는데 특히 
미국의 페더럴펀드금리선물(federal funds futures)은 시장참가자들에게 향후 미연준이 
통화정책을 어떤 방향으로 운용할 것인지를 가늠해 볼 수 있는 지표역할을 하기도 한다.
금리스왑
금리스왑(IRS; Interest Rate Swaps)이란 금리변동위험 헤지 및 차입비용 절감 등을 
위하여 거래당사자간에 원금교환 없이 정기적(3개월)으로 변동금리(91일물 CD금리)와 
고정금리(IRS금리)를 교환하는 거래를 말한다. 이때 고정금리를 지급하는 대신 변동금리
를 수취하는 거래를 금리스왑 pay거래라 하며 반대로 고정금리를 수취하는 대신 변동금
리를 지급하는 거래를 금리스왑 receive거래라 한다. 거래단위와 만기는 제한이 없지만<br>

**Node ID:** 30fe6206-761d-4eec-8620-5e136ae6b755<br>**Similarity:** 2.0985963344573975<br>**Text:** 23
ㄱ 
4.0%, 9개월후 4.5%가 된다. 자금 차입자 입장에서는 앞으로 시장금리가 상승할 것으로 
예상하는 경우에는 고정금리를 이용하는 것이 유리하고 시장금리가 하락할 것으로 예상
하는 경우에는 변동금리를 이용하는 것이 유리하다. 주요 선진국 중앙은행의 정책금리 
인상, 양적완화 축소 등 글로벌 금융긴축으로의 전환이 우리나라에도 시장금리 상승을 
야기하고 있는 상황에서 주택담보대출 차입자의 재무건전성을 유지하기 위해 정부가 
고정금리대출을 장려하는 것도 이런 이유 때문이다. 일반적으로 동일 만기에서는 고정금
리가 변동금리 보다 높은데 이는 자금 대여자(은행)에게 약정 기간중 금리 변동에 따른 
위험 프리미엄이 존재하기 때문이다 . 
 연관검색어 : 변동금리, 양도성예금증서 (CD)
고정금리부채권(SB)
고정금리부채권(Straight Bond)이란 정해진 기일에 고정된 이자를 지급하고 정해진 
만기에 원금을 지급하는 가장 일반적인 형태의 채권으로 전환사채(CB; Convertible 
Bond)의 주식전환권과 같이 특별한 조건이 없는 채권을 말한다. 회사채, 국채 등 대부분
의 채권이 고정금리부로 발행된다. 이에 대비되는 채권으로 변동금리부채권 (FRN; 
Floating Rate Note)이 있다. 이는 정해진 기일에 특정 금리(예: 국내에서는 3개월 CD금
리, 해외에서는 LIBOR금리)에 연동된 금리를 지급하고 정해진 만기에 원금을 지급하는 
채권이다. 고정금리부채권에 대한 투자는 향후 금리 하락이 예상될 때 유리하며 금리 
상승이 예상될 때에는 변동금리부채권에 투자하는 것이 유리하다. 한편 인플레이션이 
심할 때 인플레이션을 헤지하기 위한 목적으로 발행되는 채권이 물가연동채권인데 이는 
원금이 물가상승률(주로 CPI 사용)에 연동하여 증가한다는 면에서 변동금리부채권과 
구별된다.<br>

### 3. LLM 답변 생성

In [ ]:
# API KEY 설정
from dotenv import load_dotenv
load_dotenv()

In [13]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query(query)
print(response)

변동금리는 일정 주기별로 시장 금리를 반영하여 약정 금리가 변동하는 금리를 의미합니다. 예를 들어, 변동주기가 3개월인 경우, 약정 금리는 3개월마다 변동된 시장 금리에 따라 조정됩니다. 변동금리는 시장 금리가 하락할 것으로 예상될 때 유리하며, 이는 고정금리와 대비됩니다. 변동금리부채권은 특정 금리에 연동된 금리를 지급하며, 금리 상승이 예상될 때 투자하는 것이 유리합니다.
